In [ ]:
import numpy as np
import random
import math
import json
import sys
import time

from scipy.spatial import distance

In [ ]:
# try the load function

def load(file_name):
    # data(list of list): [[index, dimensions], [.., ..], ...]
    data = []              
    fh = open(file_name)
    for line in fh:
        line = line.strip().split(',')
        temp = [int(line[0])]
        for feature in line[1:]:
            temp.append(float(feature))
        data.append(temp)  
    return data


inputpath = 'data0.txt'

In [ ]:
data = load (inputpath)
data

[[0,
  67.08558097165057,
  -344.6314012666178,
  -376.1425874412542,
  37.9535038405144,
  187.06589997682897,
  -121.36987278408489,
  288.0927389844678,
  1.6906741445673745,
  34.70534638758329,
  -322.9834187315627,
  110.25171875736608,
  -232.5583034220324,
  -451.36222680752377,
  36.93407436289739,
  16.499577774076226,
  313.45899833454683,
  102.84504451381518,
  -394.5798540518628,
  -217.3427806470651,
  171.93073643356686,
  -462.48006762350195,
  -352.0003216711611,
  -160.00659843683468,
  -319.2108264867853,
  -64.66990229662326,
  264.89697981708855,
  -93.96555276760955,
  -135.2606989267505,
  320.54224045902214,
  266.49802830962346,
  170.9305852255494,
  -211.01424739453236,
  187.9105454882922,
  -252.59171753652228,
  -446.6066958535551,
  -303.7045631400592,
  228.18128611194084,
  107.49067207637583,
  -381.03544846595975,
  312.0971878367707,
  272.04586997490475,
  400.3959853763721,
  -23.441165458787815,
  -100.89435754480786,
  341.463657850407,
  -320.0

In [ ]:
n,d = np.shape(data)
np.shape(data)

(21318, 51)

In [ ]:
def initialize_centroids(data, dimension, k):
    centroids = np.zeros ((k,dimension))
    # centroids = [[0 for _ in range(dimension)] for _ in range(k)]
    max_feature_vals = [0 for _ in range(dimension)]
    min_feature_vals = [float('inf') for _ in range(dimension)]
    for point in data:
        for i in range(dimension):
            max_feature_vals[i] = max(max_feature_vals[i], point[i ])
            min_feature_vals[i] = min(min_feature_vals[i], point[i ])
    for i in range(dimension):
        min_feature_val = min_feature_vals[i]
        max_feature_val = max_feature_vals[i]
        diff = max_feature_val - min_feature_val
        for j in range(k):
            centroids[j][i] = min_feature_val + diff * random.uniform(1e-5, 1)
    print(type(centroids))
    return centroids, True

def reaffiliate_everyone( indata, clustaff, cents, k, first_run ):
  print('reaffiliating everyone...')

  n,d = np.shape (indata)

  if first_run == True:  
    # ignore clustaff input and init clustaff from scratch
    clustaff = np.array([[indata[i], None] for i in range(n)])
    print('running reaffiliation program for the first time. Initializing clustaff from scratch...')
  # else:
    # use the clustaff tht was passed into this function

  # after each reaffiliation, we'll also keep count of points per cluster
  cpcount = [0 for _ in range(k)]

  # stores whether at least one point was reaffliated. So, before reaffiliation, this value will be zero.
  flag = 0

  # counts the number of points that were reaffiliated. So, before reaffiliation, this value will also be zero.
  reaff_count = 0

  print('flag before mass reaffiliation: ', flag)
  for thispoint_index in range (len(indata)):
    # print(thispoint_index)
    thispoint = indata[thispoint_index]
    distances = []

    # find distances from thispoint to all centroids
    for cent in cents:
      distances.append ( distance.euclidean(thispoint, cent) )

    # find the smallest distance, and corresponding cluster number. 
    closest_distance = np.amin(distances)
    min_dist_index = distances.index (closest_distance)
    # print('thispoints closest centroid is centroid number ', min_dist_index)

    # thispoint will be affiliated with cluster number min_dist_index; 
    # but only if min_dist_index is different from thispoint's existing cluster number.
    if clustaff[thispoint_index][1] != min_dist_index:
      flag = 1
      reaff_count += 1
      clustaff[thispoint_index][1] = min_dist_index
      
    # else:
      # don't bother reaffiliating this point. Flag remains == 0.
      # print('this point will NOT be reaffiliated.')


    # once reaffilliation is done, count this point.
    cpcount[clustaff[thispoint_index][1]] += 1
    reaff_count += 1
  print('flag after mass reaffiliation: ', flag, 'which means...')

  if flag == 1:
    print('At least one point was reaffiliated.')
  elif flag == 0:
    print('No points were reaffiliated. Kmeans has finally converged.')
  
  print(cpcount)
  print('number of points reaffiliated:', reaff_count )

      
  return clustaff, cpcount, flag, False, reaff_count

def recompute_centroids( indata, clustaff, cpcount, k ):
  global cent
  print('recomputing centroids...')
  n,d = np.shape(indata)

  cent =  np.array([[0 for _ in range(d)] for _ in range(k)] )
  print(np.shape(cent))

  # print('before add:', cent)
  for i, aff in enumerate( clustaff ):
    # find out which cluster this point belongs to. Call it c.
    c = aff[1]
    # print('point number', i , 'belongs to cluster number', c)

    # add this point's coordinates to the cth centroid
    cent[c] = np.add ( cent[c], aff[0] ) 
    # print(cent[c])
  

  for i in range(k):
    # divide each centroid's coordinates 
    # with the number of points that belong to that centroid
    if cpcount[i] ==0:
      continue
    cent[i] = np.divide( cent[i], cpcount[i])
    
  # print('after recomputing:')
  # for centroid in cent:
  #   print(centroid)
  # print('done\n')
  print('done')

  return cent

In [ ]:
def get_sample(data):
    length = len(data)
    sample_size = int(length * 0.01)
    random_nums = set()
    sample_data = []

    for i in range(sample_size):
        random_index = random.randint(0, length - 1)
        while random_index in random_nums:
            random_index = random.randint(0, length - 1)
        random_nums.add(random_index)
        sample_data.append(data[random_index])
    return sample_data

In [ ]:
sample_data = get_sample(data)
print(np.shape(sample_data))

(213, 51)


In [ ]:
#kmeans

def kmeans( data, d, k):
    centroids_advanced, first_time  = initialize_centroids (data, d, k)
    print('centroids initialized', np.shape(centroids_advanced), '(index column included)')
    print('first_time:', first_time )
    # print(centroids_advanced)

    cluster_affiliations = []
    cluster_affiliations, cluster_point_count, flag, first_time, counter = reaffiliate_everyone( data , cluster_affiliations, centroids_advanced, k, first_time )

    #if prints nothing, then ok.
    for affiliation in cluster_affiliations:
      if affiliation[1] == None:
        print(affiliation[1])

    centroids_advanced = recompute_centroids ( data, cluster_affiliations, cluster_point_count, k )
    print(flag)
    print(first_time)

    while flag:
        cluster_affiliations, cluster_point_count, flag, first_time, counter = reaffiliate_everyone( data , cluster_affiliations, centroids_advanced, k, first_time )
        centroids_advanced = recompute_centroids ( data, cluster_affiliations, cluster_point_count, k )
        print('flag:', flag)
        
    return centroids_advanced, cluster_affiliations, cluster_point_count

In [ ]:
k = 4
centroids, cluster_affiliation, cluster_point_count = kmeans(sample_data, d, k * 3)

<class 'numpy.ndarray'>
centroids initialized (12, 51) (index column included)
first_time: True
reaffiliating everyone...
running reaffiliation program for the first time. Initializing clustaff from scratch...
flag before mass reaffiliation:  0
flag after mass reaffiliation:  1 which means...
At least one point was reaffiliated.
[13, 11, 14, 4, 20, 13, 1, 5, 20, 40, 47, 25]
number of points reaffiliated: 426
recomputing centroids...
(12, 51)
done
1
False
reaffiliating everyone...
flag before mass reaffiliation:  0
flag after mass reaffiliation:  1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


 which means...
At least one point was reaffiliated.
[24, 10, 17, 14, 18, 13, 1, 7, 15, 30, 40, 24]
number of points reaffiliated: 247
recomputing centroids...
(12, 51)
done
flag: 1
reaffiliating everyone...
flag before mass reaffiliation:  0
flag after mass reaffiliation:  1 which means...
At least one point was reaffiliated.
[28, 10, 17, 19, 17, 14, 1, 8, 15, 26, 34, 24]
number of points reaffiliated: 229
recomputing centroids...
(12, 51)
done
flag: 1
reaffiliating everyone...
flag before mass reaffiliation:  0
flag after mass reaffiliation:  1 which means...
At least one point was reaffiliated.
[29, 10, 17, 16, 17, 17, 1, 8, 15, 25, 34, 24]
number of points reaffiliated: 219
recomputing centroids...
(12, 51)
done
flag: 1
reaffiliating everyone...
flag before mass reaffiliation:  0
flag after mass reaffiliation:  1 which means...
At least one point was reaffiliated.
[29, 10, 18, 17, 17, 17, 1, 9, 13, 25, 33, 24]
number of points reaffiliated: 217
recomputing centroids...
(12, 51)
don

In [ ]:
# init global variables
k = 4 # k * 4
print(k)

n, d = np.shape(data)
print(n,d)

threshold = 4 * math.sqrt(d)

4
21318 51


In [ ]:
def get_euclidean_distance(p1, p2, p1_with_index, p2_with_index):
    i1 = 0
    i2 = 0
    if p1_with_index:
        i1 = 1
    if p2_with_index:
        i2 = 1
    sd_sum = 0
    for d in range(len(p1) - i1):
        sd_sum += (p1[d + i1] - p2[d + i2]) ** 2
    return math.sqrt(sd_sum)

In [ ]:
def gather_clusters_info(centroids, cluster_affiliation):
    clusters_temp = [[tuple(centroid), []] for centroid in centroids]
    for a in cluster_affiliation:
        features, cluster_index = a[0], a[1]
        clusters_temp[cluster_index][1].append(features)
    clusters = []
    for cluster in clusters_temp:
        if cluster[0][0] != None:
            clusters.append(cluster)
    return clusters

def delete_redundant_cluster(clusters, final_count):
    index_count = []
    for index, cluster in enumerate(clusters):
        index_count.append((len(cluster[1]), index))
    index_count.sort()  #sort with cluster size
    for i in range(len(clusters) - final_count):
        clusters.pop(0)
    return clusters

def initialize_stat(clusters, dimension, cluster_min_size):
    stats = []
    set_point_index = []
    remaining_points = []
    
    #clusters: [(centroid1, [(point1 features), (point2 features), ...]), (centroid2, ...)]
	#centroid: (centroid features)
    #points: [(point1 51 features); (point2 51 features) ... ... ..] points in the cluster
    for centroid, points in clusters:
        if len(points) >= cluster_min_size:
            #stat: [0, (tuple 50 features). (tuple 50 features)]
            stat = [0, [0 for _ in range(dimension)], [0 for _ in range(dimension)]]
            point_index = set()
            for point in points:
                point_index.add(point[0])
                stat[0] += 1
                for d in range(dimension):
                    stat[1][d] += point[d + 1]
                    stat[2][d] += point[d + 1] ** 2
                    
            # stats: [(numberofpoints in cluster0, SUM(tuple 50 features), SUMSQ (tuple 50 features)); (numberofpoints in cluster1, SUM(tuple 50 features), SUMSQ (tuple 50 features));]
            stats.append(stat)
            #set_point_index: [{point indeices in cluster 0}, {point indices in cluster 1}]
            set_point_index.append(point_index)
        else:
            remaining_points.extend(points)
    return (stats, set_point_index, remaining_points)

def get_centroids_sd(stat, dimension):
    centroids = []
    cluster_sd = []
    for N, SUM, SUMSQ in stat:
        centroid = []
        sd = []
        for d in range(dimension):
            centroid.append(SUM[d] / N)
            sd.append(math.sqrt(SUMSQ[d] / N - (SUM[d] / N) ** 2))
        centroids.append(centroid)
        cluster_sd.append(sd)
    return (centroids, cluster_sd)

def get_mahalanobis_distance(p1, p2, p1_with_index, p2_with_index, sd, dimension):
    i1 = 0
    i2 = 0
    if p1_with_index:
        i1 = 1
    if p2_with_index:
        i2 = 1
    sum_sq = 0
    for d in range(dimension):
        sum_sq += ((p1[d + i1] - p2[d + i2]) / sd[d]) ** 2
    return math.sqrt(sum_sq)

def update_stat(data, stat, set_point_index, dimension, threshold, first_load):
    #set_point_index: [{point indeices in cluster 0}, {point indices in cluster 1}]
           
   
    #centroids: [(average  50 tuple); (average  50 tuple); ... ... ]
    #cluster_sd = [(cluster0 sd 50 tuple); (cluster1 sd 50 tuple); ... ... ]
    centroids, cluster_sd = get_centroids_sd(stat, dimension)

    remaining_points = []

    for point in data:
        point = tuple(point)

        if first_load:
            point_exist = False
            for point_index in set_point_index:
                #check whether the point from data already has cluster assignments?
                if point[0] in point_index: #search the point in the list point_index for corresponding cluster 
                    point_exist = True
                    break
            if point_exist:
                continue

        min_mahalanobis_distance = float('inf')
        for index, centroid in enumerate(centroids):
            mahalanobis_distance = get_mahalanobis_distance(point, centroid, True, False, cluster_sd[index], dimension)     
            if mahalanobis_distance < min_mahalanobis_distance:
                min_mahalanobis_distance = mahalanobis_distance
                min_index = index
        if min_mahalanobis_distance < threshold:
            set_point_index[min_index].add(point[0]) #adding the new point index
            stat[min_index][0] += 1 # increase point count
            for d in range(dimension):
                stat[min_index][1][d] += point[d + 1]
                stat[min_index][2][d] += point[d + 1] ** 2
        else:
            remaining_points.append(point)
    return (stat, set_point_index, remaining_points)

def merge_clusters(stat1, point_index1, stat2, point_index2, i, j, dimension, combined_cs):
    s1 = []
    s2 = []
    for n in range(3):
        s1.append(stat1[i][n])
        s2.append(stat2[j][n])
    combined_stat = [s1[0] + s2[0]]
    v1 = []
    v2 = []
    for d in range(dimension):
        v1.append(s1[1][d] + s2[1][d])
        v2.append(s1[2][d] + s2[2][d])
    combined_stat.append(v1)
    combined_stat.append(v2)
    
    temp = point_index1[i]
    for point in point_index2[j]:
        temp.add(point)
    if combined_cs:
        del stat2[max(i, j)]
        del stat2[min(i, j)]
        del point_index2[max(i, j)]
        del point_index2[min(i, j)]
    else:
        del stat1[i]
        del stat2[j]
        del point_index1[i]
        del point_index2[j]
    stat2.append(combined_stat)
    point_index2.append(temp)

    return (stat1, point_index1, stat2, point_index2)

def check_merge_clusters(stat1, point_index1, stat2, point_index2, dimension, combined_cs, threshold):
  while True:
      merged = False
      
      if not combined_cs:
          if stat1:
              centroids1, cluster_sd1 = get_centroids_sd(stat1, dimension)
              centroids2, cluster_sd2 = get_centroids_sd(stat2, dimension)
              for i in range(len(stat1)):
                  if merged:
                      break
                  for j in range(len(stat2)):
                      mahalanobis_distance = get_mahalanobis_distance(centroids1[i], centroids2[j], False, False, cluster_sd2[j], dimension)
                      if mahalanobis_distance < threshold:
                          stat1, point_index1, stat2, point_index2 = merge_clusters(stat1, point_index1, stat2, point_index2, i, j, dimension, False)
                          merged = True
                          break
          if not merged:
              return (stat1, point_index1, stat2, point_index2)
          
      else:
          cs = stat2
          cs_length = len(cs)
          if cs_length > 1:
              centroids, cluster_sd = get_centroids_sd(cs, dimension)
              for i in range(cs_length - 1):
                  if merged:
                      break
                  for j in range(i + 1, cs_length):
                      mahalanobis_distance = get_mahalanobis_distance(centroids[i], centroids[j], False, False, cluster_sd[j], dimension)
                      if mahalanobis_distance < threshold:
                          stat1, point_index1, stat2, point_index2 = merge_clusters(cs, point_index2, cs, point_index2, i, j, dimension, True)
                          merged = True
                          break
          if not merged:
              return (stat2, point_index2)

In [ ]:
#if not enough clusters, run kmeans again
K = 4
while True:
  centroid_count = 0
  for centroid in centroids:
      if centroid[0] != None:
          centroid_count += 1
  print('centroid count:', centroid_count)
  if centroid_count < K:
      centroids, cluster_affiliation = kmeans(sample_data, dimension, K * 3) #randomize and restart kmeans until get K or more clusters
  else:
    print('kmeans done')
    break

centroid count: 12
kmeans done


In [ ]:
def gather_clusters_info(centroids, cluster_affiliation):
  clusters = []
  for i in range(k*3):
    cluster = []
    cluster.append([])
    cluster.append([])
    clusters.append(cluster)
  # clusters now has 12 rows.

  # now, let's iterate over each cluster_affiliation to gather_cluster_info.

  for aff in cluster_affiliation:
    target_cluster_number = aff[1]
    target_cluster = clusters[target_cluster_number]
    target_cluster[0] = centroids[target_cluster_number]
    
    target_cluster[1].append (aff[0])

  return clusters

In [ ]:
print(np.shape(cluster_affiliation))
print(np.shape(centroids))

(213, 2)
(12, 51)


In [ ]:
# clusters: [(centroid1, [(point1 features), (point2 features), ...]), 
# (centroid2, ...)]
# clusters: [ () ]

clusters = gather_clusters_info(centroids, cluster_affiliation)
print('cluster info gathered. shape of datastructure:',np.shape(clusters))

cluster info gathered. shape of datastructure: (12, 2)


In [ ]:
# delete redundant clutser
from heapq import nlargest

def delete_redundant_cluster(clusters,k):

  print(cluster_point_count)
  largest_counts = nlargest(k+1, cluster_point_count)

  # get the associated cluster index of each largest count
  largest_indexes = []
  for _ in range(k+1):
    largest_indexes.append( cluster_point_count.index(largest_counts[_]) )

  print('Biggest member populations are in these cluster numbers:', largest_indexes)

  for num in reversed(range(len(clusters))) :
    if num not in largest_indexes:
      print('redundant cluster deleted')
      del clusters[num]

  return clusters

In [ ]:
  # DELETE REDUNDANT CLUSTERS
  print('clusters before:', np.shape(clusters))
  clusters = delete_redundant_cluster(clusters, k)
  print('clusters after:', np.shape(clusters))

clusters before: (12, 2)
[29, 10, 23, 25, 17, 20, 1, 9, 10, 21, 24, 24]
Biggest member populations are in these cluster numbers: [0, 3, 10, 10, 2]
redundant cluster deleted
redundant cluster deleted
redundant cluster deleted
redundant cluster deleted
redundant cluster deleted
redundant cluster deleted
redundant cluster deleted
redundant cluster deleted
clusters after: (4, 2)


In [ ]:
clusters[0]

[array([7286,   60,  -38,  -11,  -68,   16,   44,  -41,  -44,   56,    6,
          48,  -15,  -68,   73,    6,   60,   63,   41,  -36, -106,  -84,
         -40,   53, -130,  -43,  -31,  -62,  -76,  -65,   -7,  159,  -27,
         110,   15,    8,  -15,  -36,   71,    8,   49,  -42,  175,  -27,
         -79,  -16,  -17,  -67,  -83,   28,  -92]),
 [[7889,
   270.4583611485292,
   -116.18390689777881,
   248.6413499581339,
   -200.59663269239832,
   -219.70677021188348,
   235.4303255220681,
   135.65966728187652,
   92.48497583897336,
   172.98584383872395,
   54.32610022811794,
   16.530712442871476,
   107.25331296000253,
   -200.6354532947392,
   114.55634300934418,
   284.58810008326356,
   320.8036373816131,
   22.522698896851736,
   323.5294529047233,
   -270.4083223641529,
   -303.56498091867763,
   -243.95324518143855,
   -97.14198585201045,
   155.50247867631032,
   -260.9659424213505,
   125.77352796122875,
   -150.68956956765186,
   -152.1565225363137,
   -40.95947304173509,


In [ ]:
# ds: [(number of points N in cluster 0, [SUM], [SUMSQ]), (number of points N in cluster 1, [SUM], [SUMSQ]), ...] 
  # ds_point_index:[{cluster0 point indices}, {cluster1 point indices}, .....]

#ds, ds_point_index, temp = initialize_stat(clusters, dimension, 1) #based on preliminary clustering

# init ds and ds_indexes

n,d = np.shape(data)
print(n,d)

ds = []
for i in range(k):
  temp = [ 0, np.zeros(d), np.zeros(d) ]
  ds.append(temp)

for d in ds:
  print(d)

ds_point_index = [[] for _ in range(k) ]
print(ds_point_index)

21318 51
[0, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]
[0, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]
[0, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
print(np.shape(clusters))

for cluster in clusters:
  print(cluster[1])

(4, 2)
[[7889, 270.4583611485292, -116.18390689777881, 248.6413499581339, -200.59663269239832, -219.70677021188348, 235.4303255220681, 135.65966728187652, 92.48497583897336, 172.98584383872395, 54.32610022811794, 16.530712442871476, 107.25331296000253, -200.6354532947392, 114.55634300934418, 284.58810008326356, 320.8036373816131, 22.522698896851736, 323.5294529047233, -270.4083223641529, -303.56498091867763, -243.95324518143855, -97.14198585201045, 155.50247867631032, -260.9659424213505, 125.77352796122875, -150.68956956765186, -152.1565225363137, -40.95947304173509, -307.9582495559163, -88.64325894950895, 278.1799303073749, -23.793017143683876, 324.86360263041666, 76.17415375604457, -175.6852727543923, 277.70322818837667, 193.95766852473608, 185.31772191185485, 88.04907162538494, 82.53094577304263, -49.80690511155107, 331.25804453511824, 277.2496042247063, 65.99347612319039, -37.66646817704891, -157.96078145810992, -239.0094732784887, -177.10953857743496, 282.25977574913713, -286.1884

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
#populate ds and ds_indexes using clusters
n, dimension = np.shape(data)

for i,d in enumerate(ds):
  print('updating ds for cluster number', i ,'...')

  for j, cluster in enumerate(clusters):
    this_cluster_members = np.array(cluster[1])

    # how many members does this cluster have?
    # increase N that many times
    this_cluster_size = len (this_cluster_members)
    d[0] += this_cluster_size

    for this_cluster_member in this_cluster_members:
      #update sum
      # print(np.shape(d[1]))
      # print(np.shape(this_cluster_member))
      d[1] = np.add (d[1] , this_cluster_member)
      d[2] = np.add (d[1] , this_cluster_member**2)

      ds_point_index[i].append (this_cluster_member[0])

  print('done')

updating ds for cluster number 0 ...
done
updating ds for cluster number 1 ...
done
updating ds for cluster number 2 ...
done
updating ds for cluster number 3 ...
done


In [ ]:
print(ds[0])

[101, array([ 9.47537000e+05,  6.10014849e+03, -6.33433578e+03, -3.37808332e+03,
       -3.16625693e+03,  2.97675022e+03,  4.09944654e+02, -2.06854575e+03,
       -2.90531276e+03,  3.94410331e+03, -2.07346614e+03,  3.45128067e+03,
       -5.92170532e+02, -6.53266923e+03,  5.33495670e+03, -9.93805527e+01,
        8.25225612e+03,  4.20720155e+03,  6.96174284e+02, -3.19423023e+03,
       -7.09147940e+03, -1.03559932e+04, -4.76127850e+03,  4.01818509e+03,
       -1.22144661e+04, -5.30904190e+03, -1.00473884e+03, -6.23621391e+03,
       -6.74721399e+03, -2.34288578e+03,  1.54016662e+03,  1.26075457e+04,
       -1.33824350e+03,  1.12617577e+04, -3.02696725e+03, -2.11698157e+03,
       -6.25406640e+03, -3.21068739e+03,  7.59182954e+03, -1.43082397e+03,
        6.43045619e+03, -7.73727134e+02,  1.66591475e+04, -2.46690285e+03,
       -5.98658561e+03, -9.02280007e+02, -4.08017511e+03, -4.86207687e+03,
       -7.72041949e+03,  2.92090563e+03, -4.04658914e+03]), array([ 1.38470066e+08,  7.5040608

In [ ]:
first_load = True

#init cs
n,d = np.shape(data)
print(n,d)

cs = []
for i in range(k):
  temp = [ 0, np.zeros(d), np.zeros(d) ]
  ds.append(temp)

for d in ds:
  print(d)

cs_point_index = [[] for _ in range(k) ]
print(ds_point_index)

# init rs
rs = []
inter_results = []

In [ ]:
def compute_dimensional_variances ( inset ):

  inset_variances = []

  for i,cluster_summary in enumerate(inset):
    this_N = cluster_summary[0]
    this_sum = cluster_summary[1]
    this_sumsq = cluster_summary[2]

    this_variances = (this_sumsq/this_N)-(this_sum/this_N)**2
    inset_variances.append(this_variances)

  inset_variances = np.vstack(inset_variances)
  
  return inset_variances

def mahadist ( inpoint_features, centroid, inset_variances ):
  # step1: normalize in each dimension
  sum = 0
  try:
    for i in range (0,d-1):
    # for each dimension:
      # 1. Normalize
      xi = inpoint_features[i]
      ci = (centroid[i])
      sigma_i = inset_variances[i]

      yi_norm = ( (xi - ci) / sigma_i)**2

      # 2. Sum
      sum += yi_norm
  except:
    return 2

  # 3. square 
  ans = math.sqrt(sum)

  return ans


def update_stat ( indata, inset, inset_point_index, dim, thres, first_load):
  remaining_points = []

  # compute each cluster's variances in each dimension
  inset_variances = compute_dimensional_variances(inset)

  # for each cluster_summary in inset:
  for i in range(len(inset)):
    this_centroid = inset[i][1] / inset[i][0]

    for datapoint in indata:
      if mahadist ( this_centroid, datapoint, inset_variances ) < threshold:
        # merge datapoint into inset[i]
        
        #N
        inset[i][0] +=1
        inset_point_indexes[i].append(datapoint[0])

        #sum
        inset[i][1] = np.add ( inset[i][1], datapoint )

        #sumsq
        inset[i][2] = np.add ( inset[i][2], datapoint**2 )
        
      else: # add datapoint[0] into remaining_points
        remaining_points.apppend ( datapoint[0] )



  return inset, inset_point_indexes, remaining_points

In [ ]:
# just to check this is the last chunk of data

# try:
#     data = load(inputpath + 'data' + str(data_num) + '.txt')
          
# except:
#     cs, cs_point_index, ds, ds_point_index = check_merge_clusters(cs, cs_point_index, ds, ds_point_index, dimension, False, threshold)
#     break

In [ ]:
 # assign points to ds
  ds, ds_point_index, remaining_points = update_stat(data, ds, ds_point_index, dimension, threshold, first_load) #assign new points to ds 

In [ ]:
def check_merge_clusters( inset, inset_point_index, dimension, threshold ):
  # coordinates of each centroids
  centroids = []

  # pairs of cluster numbers that need to be merged
  merge_pairs = []

  # compute each centroid for comparison
  for cluster_summary in inset:
    this_centroid = inset[i][1] / inset[i][0]
    centroids.append(this_centroid)


  cs_variances = compute_dimensional_variances(cs)

  for i, centroid_i in enumerate(centroids):
    for j, centroid_j in enumerate(centroids):
      
      # don't compare same centroid
      if i == j:
        continue
      
      if mahadist ( centroids[i], centroids[j], cs_variances) < threshold:

        # merge j into i

        # merge N
        inset[i][0] += inset[j][0]
        # merge sum
        inset[i][1] += inset[j][1]
        # merge sumsq
        inset[i][2] += inset[j][2]
        # merge count
        for that_point_index in inset_point_index[j]:
          inset_point_index[i].append(that_point_index)
        
        # delete all summaries and point indexes associated with cluster j
        del inset_point_index[j]        
        del inset[j]

  return inset,

In [ ]:
n, dimension = np.shape(data)

In [ ]:
# if cs is not empty:
if len(cs)!=0:
  # check if cs can be merged
  cs, cs_point_index = check_merge_clusters ( cs, cs_point_index, dimension, threshold )

# assign points to cs
cs, cs_point_index, remaining_points = update_stat( data, cs, cs_point_indexes, dimension, threshold, )

In [ ]:
while True:

      #remaining_points: [{point tuple}, {point tuple}, .....]
  if first_load:
      first_load = False  
  if remaining_points:    
      if cs:
          # merge cs if needed
              # cs: [(number of points N in CScluster 0, [SUM], [SUMSQ]), (number of points N in CS cluster 1, [SUM], [SUMSQ]), ...] 
              # cs_point_index:[{CScluster0 point indices}, {CScluster1 point indices}, .....]
          cs, cs_point_index = check_merge_clusters(cs, cs_point_index, cs, cs_point_index, dimension, True, threshold)
          # assign points to cs
          cs, cs_point_index, remaining_points = update_stat(remaining_points, cs, cs_point_index, dimension, threshold, False)
      
      centroids, cluster_affiliation = kmeans(remaining_points + rs, dimension, 3 * K)
      clusters = gather_clusters_info(centroids, cluster_affiliation)
      cs_temp, cs_point_index_temp, rs = initialize_stat(clusters, dimension, 2)
      cs.extend(cs_temp)
      cs_point_index.extend(cs_point_index_temp)
  data_num += 1
  ds_point_count = 0
  cs_point_count = 0
  inter_results.append((data_num, len(ds), sum([len(points) for points in ds_point_index]), len(cs), sum([len(points) for points in cs_point_index]), len(rs)))

In [ ]:
results = {}
for index, points in enumerate(ds_point_index):
    for point in points:
        results[str(point)] = index
for points in cs_point_index:
    for point in points:
        results[str(point)] = -1
for point in rs:
    results[str(point[0])] = -1

In [ ]:
fh = open(output1, 'w')
json.dump(results, fh)
fh.close()

fh = open(output2, 'w')
fh.write('round_id,nof_cluster_discard,nof_point_discard,nof_cluster_compression,nof_point_compression,nof_point_retained')
for line in inter_results:
    fh.write('\n')
    fh.write(str(line).strip('()'))
fh.close()	

In [ ]:
print('Duration: %s' % (time.time() - start))

In [ ]:
def compute_average_intra_cluster_distances( cent, clustaff, k ):

  avg_distances = np.zeros ((k))

  # a. first, count how many points are in each cluster
  clusterpoint_counts = np.zeros(k)
  for aff in clustaff:
    point_affiliation = aff[1]
    clusterpoint_counts[point_affiliation] += 1

  # b. then, perform n sum operations 
  for i, aff in enumerate(clustaff):
    # print('checking point number', i)
    thispoint_coordinates = aff[0]
    c = aff[1]
    thispoint_distance_to_centroid = distance.euclidean ( thispoint_coordinates[1:], cent[c][:50] )

    # print(i, 'this points distance to affiliated centroid number:', c,'. Distance:' ,thispoint_distance_to_centroid)

    # add this distance to entry number c in average_intra_cluster_distances
    avg_distances[c] = avg_distances[c] + thispoint_distance_to_centroid
    # print(avg_distances[c])

  # c. then, perform k divide operations
  for i, thiscount in enumerate(clusterpoint_counts):
    avg_distances[i] = np.divide ( avg_distances[i] , thiscount )

  return avg_distances

def db_index ( cent, clustaff, k):
  print('calculating db index...')

  # print('cent:', cent)
  # print('clustaff:', clustaff[10])

  # 1. initialize and populate average intra cluster distances array
  
  average_intra_cluster_distances = compute_average_intra_cluster_distances (cent, clustaff, k)
  # print('initialized and populated average intra_cluster distances:', np.shape(average_intra_cluster_distances))

  # 2. initialize and populate inter_centroid_distances array

  inter_centroid_distances = np.zeros ((k,k))
  for i in range (k):
    for j in range(k):
      inter_centroid_distances[i][j] = distance.euclidean ( cent[i], cent[j] )
  # print('initialized inter_centroid_distances:', np.shape(inter_centroid_distances))

  presum_array = []
  for i in range(k):
    a_matrix = []
    for j in range(k):
      if i == j:
        continue
      #else, calculate a for (i,j)
      a = average_intra_cluster_distances[i] - average_intra_cluster_distances[j]
      a = a / inter_centroid_distances[i][j]
      a_matrix.append(a)
    
    # for current i, append max value of a into presum_array
    presum_array.append ( max(a_matrix) )

  ans = np.sum(presum_array)
  ans = ans/k
  return ans